In [ ]:
!unzip dip_dataset.zip

In [ ]:
!pip uninstall ultralytics

In [ ]:
import ultralytics

ultralytics.checks()

In [ ]:
from ultralytics.data.utils import autosplit

autosplit(path = 'images', weights = (0.8, 0.1, 0.1))

In [ ]:
from natsort import natsorted, ns
import os
import shutil
from skimage.transform import resize, rescale
from skimage.io import imsave, imread
import numpy as np
import cv2
from pathlib import Path

classes = ['capacitor', 'potentiometer', 'led', 'diode', 'resistor', 'integrated_circuit', 'transistor', 'switch', 'inductor', 'jumper']

fil = 'test'

for classe in classes:
    os.makedirs(os.path.join(fil, classe), exist_ok=True)

with open('autosplit_' + fil + '.txt') as f:
    for line in f:
        line = line[:-1]
        dest = line.replace('images', 'datasets/dip/'+fil)
        shutil.copy(line, dest)
        try:
          shutil.copy(line[:-3]+'txt', dest[:-3]+'txt')
        except:
          try:
            shutil.copy(line[:-8]+'.txt', dest[:-3]+'txt')
          except:
            shutil.copy(line[:-9]+'.txt', dest[:-3]+'txt')

In [ ]:
import pathlib

path_root = pathlib.Path('./')
path_imgs = pathlib.Path('./images')

In [ ]:
import skimage as ski
import numpy as np
import cv2

N = 9
kernel = np.ones(N)/(N)
kernel = np.reshape(kernel, (3, 3))

for folder in path_imgs.iterdir():
  for item in folder.iterdir():
    if item.suffix == '.txt':
      continue
    image = ski.io.imread(item)

    c = 255 / np.log(1 + np.max(image))
    log_image = c * (np.log(image + 1))
    exp_image = (np.exp(image / c) - 1)

    log_image = np.array(log_image, dtype = np.uint8)
    exp_image = np.array(exp_image, dtype = np.uint8)
    mean_image = cv2.filter2D(image, ddepth=-1, kernel=kernel)

    log_image = cv2.cvtColor(log_image, cv2.COLOR_RGB2BGR)
    exp_image = cv2.cvtColor(exp_image, cv2.COLOR_RGB2BGR)
    mean_image = cv2.cvtColor(mean_image, cv2.COLOR_RGB2BGR)

    cv2.imwrite(os.path.join('images', folder.name, item.name[:-4]+'_log'+'.png'), log_image)
    cv2.imwrite(os.path.join('images', folder.name, item.name[:-4]+'_exp'+'.png'), exp_image)
    cv2.imwrite(os.path.join('images', folder.name, item.name[:-4]+'_mean'+'.png'), mean_image)

In [ ]:
# prompt: setup training for a classifier model using yolov8

from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data='dip.yaml', epochs=12, imgsz=640) # train the model


In [ ]:
from ultralytics import YOLO

model = YOLO('runs/classify/train5/weights/best.pt')

In [ ]:
!zip -r train.zip runs/classify/train5